[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mgalbright/online_purchase_optimizer/blob/main/Notebook.ipynb)

# Intro

This notebook demonstrates how to use the [online purchase optimizer](https://github.com/mgalbright/online_purchase_optimizer) library to minimize the costs of your online purchases. 

The first part of this notebook can also be run in google colab.

## Optional: install libary
Run this e.g. if running this notebook from google colab


In [ ]:
! pip install git+https://github.com/mgalbright/online_purchase_optimizer

In [1]:
import numpy as np
import pandas as pd
from online_purchase_optimizer.retailoptimizer import RetailProblem, print_avail_solvers
from pulp import LpMinimize, LpProblem, LpStatus, lpSum, LpVariable, LpInteger, LpBinary, makeDict

In [ ]:
#Optional: test your installation of Pulp optimization libary

import pulp
pulp.pulpTestAll()

## Small sample problem using Python 

For a concrete example, assume a consumer wants to purchase fishing lures online. In the example below, the consumer wishes to buy 3 packs of plastic minnows and 20 fish hooks, and is considering 2 retailers. Let's find the cheapest purchase.

In the following python code, we create variables to define the problem. We must specify:
1. Items: What items we want to buy? (I.e. what fishing lures to buy?)
1. Quantities: How many of each item we want to buy? (E.g. we want to buy 20 hooks.)
1. Retailers: What are the names of the retailers we are shopping from?
1. Prices: What are the prices (per lure) for each fishing lure at each retailer?
1. Inventory: How many of each lure are in stock at each retailer?
   (If there is no inventory limit, enter a large value greater than the quantity desired, e.g. 100.)
1. Shipping: how much does shipping cost at each retailer, if your order is below the threshold for free shipping? (E.g., you must pay \\$4.00 for shipping at Retailer 2, unless you qualify for free shipping.)
1. Free shipping threshold: enter the thresholds (in dollars) to qualify for free shipping. (E.g. if your order at Retailer 2 is at least \\$60, you get free shipping.)
1. Sometimes you can reduce your bill by ordering more items than desired, to qualify for free shipping. Is that ok?

In [2]:
#Problem specification

#Order info
lures = ["plastic minnows", "hooks"]   
num_lures_to_buy = [3, 20]

#retailer info
retailers = ["Retailer1", "Retailer2"]  
prices = [
    [4.99, 5.49],   #price of plastic minnows at [Retailer1, Retailer2]
    [3.99, 3.49]    #price of hooks at [Retailer1, Retailer2]
]
inventory = [
    [100, 10],    #max number of plastic minnows available at [Retailer1, Retailer2]
    [15, 30],     #max number of hooks available at [Retailer1, Retailer2]
]

shipping = [7.0, 4.0] #shipping price in dollars at [Retailer1, Retailer2]
free_shipping_threshold = [50.0, 60.0] #Order threshold in dollars to qualify for free shipping @ [Retailer1, 2]

CAN_BUY_EXTRA_LURES_IF_CHEAPER = True

In [3]:
print("Available optimization solvers are: ")
print_avail_solvers()

Available optimization solvers are: 
PULP_CBC_CMD
COIN_CMD


In [ ]:
#Specify solver to use

SOLVER_NAME = 'PULP_CBC_CMD'
# SOLVER_NAME = 'GLPK_CMD'

#Create Optimization Problem

p1 = RetailProblem(lures, num_lures_to_buy, retailers, prices,
                   inventory, shipping, free_shipping_threshold,
                   solver_name=SOLVER_NAME, 
                   can_buy_extra_lures_if_cheaper = CAN_BUY_EXTRA_LURES_IF_CHEAPER)

In [5]:
#Find optimimal solution
p1.solve()

solver = PULP_CBC_CMD
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/michael/InstalledSoftware/anaconda3/envs/delete/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/p6/4vh57wr97x145dy_65rhg3k80000gn/T/61f18e2504e7477d8931d9b37a82cb5c-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/p6/4vh57wr97x145dy_65rhg3k80000gn/T/61f18e2504e7477d8931d9b37a82cb5c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 60 RHS
At line 71 BOUNDS
At line 80 ENDATA
Problem MODEL has 10 rows, 8 columns and 22 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 84.77 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 5 rows, 7 columns (7 integer (3 of which binary)) and 15 elements
Cutoff increment increased from 1e-05 to 0

In [6]:
if p1.model.status == 1:
  print(f"Optimal solution found.\n")
  p1.print_optimization_results()
else:
  print(f"WARNING: optimal solution was not found. Model status was {LpStatus[p1.model.status]}")

Optimal solution found.

Status: Optimal
empty_order_retailer1 = 1.0
empty_order_retailer2 = 0.0
pay_shipping_retailer1 = 0.0
pay_shipping_retailer2 = 0.0
quant_hooks_retailer1 = 0.0
quant_hooks_retailer2 = 20.0
quant_plastic_minnows_retailer1 = 0.0
quant_plastic_minnows_retailer2 = 3.0
Total purchase cost =  86.27000000000001


## Easily view result details
We can easily render results via dataframes generated when you call p1.solve().  
1. The dataframe df_quantities stores the optimal quantity of lures to order from each retailer.
2. The dataframe df_bills shows the bill in dollars paid to each retailer, broken down by total bill (items costs + shipping costs), shipping_bill, and items bill

In [7]:
p1.df_quantities

,retailer1,retailer2,total_number
plastic-minnows,0,3,3
hooks,0,20,20


In [8]:
p1.df_bills

,retailer1,retailer2,total
total_bill,0.0,86.27,86.27
shipping_bill,0.0,0.00,0.00
item_bill,0.0,86.27,86.27


### Result discussion
In the results above (for this simple problem), we see it is cheapest to order everything from Retailer2, i.e. order 20 hooks and 3 plastic minnows from Retailer 2. We order nothing from Retailer 1. We also see that we do not need to pay for shipping. The total bill is \\$86.27

We also include a function to output order quantities as a dict of integers

In [10]:
order_quantities = p1.convert_model_variable_quantities_to_dict()
order_quantities

{'plastic-minnows': {'retailer1': 0, 'retailer2': 3},
 'hooks': {'retailer1': 0, 'retailer2': 20}}

In [11]:
order_quantities['plastic-minnows']['retailer2']

3

You can also directly access the quanties of items to order through Pulp LpVariable objects.

In [12]:
p1.quantity_to_order['hooks']['retailer1'].varValue

0.0

In [13]:
p1.quantity_to_order['hooks']['retailer2'].varValue

20.0

## Excel example
For every-day-use, it's easier to enter data in a spreadsheet than in Python data structures, so we illustrate how to solve the same problem using the sample Excel file included in the repository. You can also directly process the file from the command line.  The Excel file examples/sample-order-small.xlsx (loaded below) includes the same information as the first example in this notebook, organized by sheets. For example, the first sheet specifies the number of lures we wish to order. Other sheets specify other information, such as prices, inventory, and shipping.
![number of items to buy](docs/imgs/small_num_items_to_buy.png)

## Google collab note:
The example below won't work in google colab out of the box. It's designed for people running the notebook locally on their computer from a clone of the [project repository](https://github.com/mgalbright/online_purchase_optimizer).  To make it work in google colab, you'll need to upload sample Excel files to your google colab space and edit the file paths below to point to the Excel files.

In [ ]:
EXCEL_INPUT_FILE = "examples/sample-order-small.xlsx"
EXCEL_OUTPUT_FILE = "examples/sample-order-small_results.xlsx"

p2 = RetailProblem.load_from_excel(EXCEL_INPUT_FILE)

In [17]:
p2.initialize_optimization_problem()

/Users/michael/InstalledSoftware/anaconda3/envs/delete/lib/python3.11/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [18]:
p2.solve()

solver = None
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/michael/InstalledSoftware/anaconda3/envs/delete/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/p6/4vh57wr97x145dy_65rhg3k80000gn/T/43573be487734aa0a499ddf41d7511f0-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/p6/4vh57wr97x145dy_65rhg3k80000gn/T/43573be487734aa0a499ddf41d7511f0-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 60 RHS
At line 71 BOUNDS
At line 80 ENDATA
Problem MODEL has 10 rows, 8 columns and 22 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 84.77 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 5 rows, 7 columns (7 integer (3 of which binary)) and 15 elements
Cutoff increment increased from 1e-05 to 0.00999
Cbc00

In [19]:
if p2.model.status == 1:
  print("Optimal solution found.\n") 

  p1.print_optimization_results()
  
  print(f"\nSaving results to {EXCEL_OUTPUT_FILE}")
  p2.save_results_to_excel(EXCEL_OUTPUT_FILE)
else:
  print(f"WARNING: optimal solution was not found. Model status was {LpStatus[p2.model.status]}")

Optimal solution found.

Status: Optimal
empty_order_retailer1 = 1.0
empty_order_retailer2 = 0.0
pay_shipping_retailer1 = 0.0
pay_shipping_retailer2 = 0.0
quant_hooks_retailer1 = 0.0
quant_hooks_retailer2 = 20.0
quant_plastic_minnows_retailer1 = 0.0
quant_plastic_minnows_retailer2 = 3.0
Total purchase cost =  86.27000000000001

Saving results to examples/sample-order-small_results.xlsx


We can see the results in the generated Excel file. They look like this:  
![number of lures to order](docs/imgs/small_number_of_lures_to_order.png)

You can also view the results directly in the notebook:

In [20]:
p2.df_quantities

,retailer1,retailer2,total_number
plastic-minnows,0,3,3
hooks,0,20,20


In [21]:
p2.df_bills

,retailer1,retailer2,total
total_bill,0.0,86.27,86.27
shipping_bill,0.0,0.00,0.00
item_bill,0.0,86.27,86.27
